# Get data

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')
checkpoints = '/content/drive/MyDrive/colab_files/imagenet64/'
if not os.path.exists(checkpoints):
    os.makedirs(checkpoints)

Mounted at /content/drive


In [2]:
import os
import shutil
!pip install unrar
if not os.path.exists('imagenet64'):
  print("Copying to local runtime...")
  shutil.copy(checkpoints + 'DATA.rar', './DATA.rar')
  print("Uncompressing...")
  !unrar x DATA.rar
print("Data ready!")

流式输出内容被截断，只能显示最后 5000 行内容。
Extracting  train/n01560419/n01560419_16395.JPEG                          80%  OK 
Extracting  train/n01560419/n01560419_16416.JPEG                          80%  OK 
Extracting  train/n01560419/n01560419_16446.JPEG                          80%  OK 
Extracting  train/n01560419/n01560419_16454.JPEG                          80%  OK 
Extracting  train/n01560419/n01560419_1647.JPEG                           80%  OK 
Extracting  train/n01560419/n01560419_16477.JPEG                          80%  OK 
Extracting  train/n01560419/n01560419_16483.JPEG                          80%  OK 
Extracting  train/n01560419/n01560419_16484.JPEG                          81%  OK 
Extracting  train/n01560419/n01560419_1658.JPEG                           81%  OK 
Extracting  train/n01560419/n01560419_16582.JPEG                          81%  OK 
Extracting  train/n01560419/n01560419_16585.JPEG     

In [3]:
%load_ext tensorboard
import datetime
import numpy as np
import os
import tensorflow as tf
import tensorflow

import matplotlib.pyplot as plt
from tqdm import tqdm

from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout

# Origional Algorithm With out any changing/ data augrumenting etc

In [4]:
# AlexNet model
class AlexNet(Sequential):
    def __init__(self, input_shape, num_classes):
        super().__init__()

        self.add(Conv2D(96, kernel_size=(11,11), strides= 4,
                        padding= 'valid', activation= 'relu',
                        input_shape= input_shape,
                        kernel_initializer= 'he_normal'))
        self.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
                              padding= 'valid', data_format= None))

        self.add(Conv2D(256, kernel_size=(5,5), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))
        self.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
                              padding= 'valid', data_format= None))

        self.add(Conv2D(384, kernel_size=(3,3), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))

        self.add(Conv2D(384, kernel_size=(3,3), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))

        self.add(Conv2D(256, kernel_size=(3,3), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))

        self.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
                              padding= 'valid', data_format= None))

        self.add(Flatten())
        self.add(Dense(4096, activation= 'relu'))
        self.add(Dense(4096, activation= 'relu'))
        self.add(Dense(1000, activation= 'relu'))
        self.add(Dense(num_classes, activation= 'softmax'))

        self.compile(optimizer= tf.keras.optimizers.Adam(0.001),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

In [5]:
num_classes = 20
model = AlexNet((227, 227, 3), num_classes)

In [6]:
# training parameters
EPOCHS = 10
BATCH_SIZE = 128
image_height = 227
image_width = 227
train_dir = "/content/train"
valid_dir = "/content/val"
model_dir = "/my_model.h5"

In [7]:
train_datagen = ImageDataGenerator(
                  rescale=1,
                  rotation_range=0,
                  width_shift_range=0,
                  height_shift_range=0,
                  shear_range=0,
                  zoom_range=0)

train_generator = train_datagen.flow_from_directory(train_dir,
                          target_size=(image_height, image_width),
                          color_mode="rgb",
                          batch_size=BATCH_SIZE,
                          seed=1,
                          shuffle=True,
                          class_mode="categorical")

valid_datagen = ImageDataGenerator(rescale=1)
valid_generator = valid_datagen.flow_from_directory(valid_dir,
                        target_size=(image_height, image_width),
                        color_mode="rgb",
                        batch_size=BATCH_SIZE,
                        seed=7,
                        shuffle=True,
                        class_mode="categorical"
                        )
train_num = train_generator.samples
valid_num = valid_generator.samples

Found 26000 images belonging to 20 classes.
Found 1000 images belonging to 20 classes.


# Run test

In [8]:
log_dir="./logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
callback_list = [tensorboard_callback]

# start training
model.fit(train_generator,
        epochs=EPOCHS,
        steps_per_epoch=train_num // BATCH_SIZE,
        validation_data=valid_generator,
        validation_steps=valid_num // BATCH_SIZE,
        callbacks=callback_list,
        verbose=0)
model.summary()

# save the whole model
model.save(model_dir)

Model: "alex_net"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 55, 55, 96)        34944     
                                                                 
 max_pooling2d (MaxPooling2  (None, 27, 27, 96)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 27, 27, 256)       614656    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 13, 13, 256)       0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 13, 13, 384)       885120    
                                                                 
 conv2d_3 (Conv2D)           (None, 13, 13, 384)       132

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Accuracy: top 1 error and top 5 error

In [9]:
class_names = os.listdir()

x_valid = []
label_batch  = []
pop = 0
t1=0
t3=0
k=0

for x_valid_b, y_valid_b in valid_generator:
  if(k > 1000/128):break
  k+=1
  prediction_values = model.predict(x_valid_b,batch_size=x_valid_b.shape[0])
  for i in range(len(prediction_values)):
    pop +=1
    #print(type(prediction_values[i]))
    answer = np.where(y_valid_b[i] == 1)[0][0]
    sorted_indices = np.argsort(prediction_values[i])[::-1]
    top_3 = sorted_indices[:3]

    top_one = sorted_indices[:1]
    if answer in top_3:t3 += 1
    if top_one == answer: t1 +=1










print(pop)


print("top 1 accuracy = %5.3f" %(t1/pop) )
print("top 5 accuracy = %5.3f" %(t3/pop) )

1/1 [==============================] - 1s 1s/step
1000
top 1 accuracy = 0.493
top 5 accuracy = 0.778
